### 영화 리뷰 데이터를 이용한 추천 시스템 구현
 * 아이템 기반 협업 필터링
    * 유사도 지표 : 상관계수 및 코사인 유사도 사용
 * 데이터 셋
    * 다운로드 : https://grouplens.org/datasets/movielens/
        * 파일명 : ml-10m.zip
    * 데이터 설명 : file:///C:/Users/toto/Documents/Github/MachineLearning_Basic_Class/data/grouplens/ml-10m/ml-10M100K/README.html

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.decomposition import TruncatedSVD

In [7]:
import os 
print("현재 위치 : ", os.getcwd() )

현재 위치 :  C:\Users\toto\Documents\Github\MachineLearning_Basic_Class\class_code


In [6]:
col_name = ["user_id", "movie_id", "rating", 'timestamp']
ratings = pd.read_csv("../../data/grouplens_ml_10m/ratings.dat", 
                      sep="::", names=col_name)
print(ratings.shape)

<ipython-input-6-bf7d8bf9304e>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv("../../data/grouplens_ml_10m/ratings.dat",


(10000054, 4)


In [9]:
col_name = ["movie_id", "movie_title", "category"]
movies = pd.read_csv("../../data/grouplens_ml_10m/movies.dat", 
                      sep="::", names=col_name, encoding='latin-1')

print(movies.shape)

(10681, 3)


<ipython-input-9-be04bd604c03>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv("../../data/grouplens_ml_10m/movies.dat",


* 총 리뷰 데이터 : 약 1000만건, 영화 정보 : 10681편

In [10]:
movie_names = movies[['movie_id', 'movie_title']]
c_movies_data = pd.merge(ratings, movie_names, on='movie_id')
print(c_movies_data.shape)
c_movies_data.head()

(10000054, 5)


,user_id,movie_id,rating,timestamp,movie_title
0,1,122,5.0,838985046,Boomerang (1992)
1,139,122,3.0,974302621,Boomerang (1992)
2,149,122,2.5,1112342322,Boomerang (1992)
3,182,122,3.0,943458784,Boomerang (1992)
4,215,122,4.5,1102493547,Boomerang (1992)


In [12]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000054 entries, 0 to 10000053
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   user_id    int64  
 1   movie_id   int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 305.2 MB


In [13]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10681 entries, 0 to 10680
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movie_id     10681 non-null  int64 
 1   movie_title  10681 non-null  object
 2   category     10681 non-null  object
dtypes: int64(1), object(2)
memory usage: 250.5+ KB


In [11]:
rating_crosstab = c_movies_data.pivot_table(values='rating', 
                                            index='user_id', 
                                            columns='movie_title', fill_value=0)
rating_crosstab.head()

movie_title,"""Great Performances"" Cats (1998)",'Round Midnight (1986),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),*batteries not included (1987),...All the Marbles (a.k.a. The California Dolls) (1981),...And God Created Woman (Et Dieu... crÃ©a la femme) (1956),...And God Spoke (1993),...And Justice for All (1979),...,Zorba the Greek (Alexis Zorbas) (1964),"Zorro, the Gay Blade (1981)",Zulu (1964),Zus & Zo (2001),[Rec] (2007),eXistenZ (1999),ffolks (a.k.a. North Sea Hijack) (1980),loudQUIETloud: A Film About the Pixies (2006),xXx: State of the Union (2005),"Ãge d'or, L' (1930)"
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
X = rating_crosstab.T
print(X.shape)

(10676, 69878)


In [15]:
SVD = TruncatedSVD(n_components=12, random_state=5)
resultant_matrix = SVD.fit_transform(X)
resultant_matrix.shape

(10676, 12)

In [16]:
### correlation matrix
corr_mat = np.corrcoef(resultant_matrix)
print( corr_mat.shape )
corr_mat

(10676, 10676)


array([[1.        , 0.91104477, 0.43040745, ..., 0.58091296, 0.51191829,
        0.86042399],
       [0.91104477, 1.        , 0.28779129, ..., 0.7131907 , 0.4264758 ,
        0.9175588 ],
       [0.43040745, 0.28779129, 1.        , ..., 0.05476156, 0.23606898,
        0.11309555],
       ...,
       [0.58091296, 0.7131907 , 0.05476156, ..., 1.        , 0.45256165,
        0.8176457 ],
       [0.51191829, 0.4264758 , 0.23606898, ..., 0.45256165, 1.        ,
        0.43937709],
       [0.86042399, 0.9175588 , 0.11309555, ..., 0.8176457 , 0.43937709,
        1.        ]])

In [18]:
col_idx = rating_crosstab.columns.get_loc("'burbs, The (1989)")
corr_specific = corr_mat[col_idx]    # Star Wars (1977)의 위치 행 획득
print(corr_specific.shape) 

(10676,)


In [19]:
result = pd.DataFrame({'corr_specific':corr_specific, 
                       'Movies': rating_crosstab.columns})

print(result.shape)
result.sort_values('corr_specific', ascending=False).head(10)

(10676, 2)


,corr_specific,Movies
3,1.000000,"'burbs, The (1989)"
3065,0.982438,European Vacation (1985)
4195,0.976229,Harry and the Hendersons (1987)
8332,0.975343,Scrooged (1988)
10376,0.974636,Who's Harry Crumb? (1989)
2005,0.973486,Clue (1985)
3656,0.973419,Funny Farm (1988)
10262,0.973254,Weird Science (1985)
9548,0.971175,Three Amigos (1986)
6192,0.968956,Meatballs Part II (1984)
